# Coiled Failcase for using xhistogram on cmip6 data

## What am I trying to achieve here

I would like to compute a joint distribution (2D histogram) of ocean volume based on values of temperature `thetao` and salinity `so`. 
This requires to take the histogram over the 3 spatial dimensions of e.g. ocean model output (`x`, `y`, and `lev`), preserving the `time` dimension.

The following function is a simple wrapper around `histogram` that provides some inputs (with hardcoded names used for my particular dataset).

Importantly the histogram is weighted by the volume of the cells, so it represents the total volume of the ocean found in a certain bin-combo of temperature and salinity.

In [1]:
from dask_gateway import GatewayCluster

cluster = GatewayCluster()
cluster.scale(10)
cluster

In [2]:
client = cluster.get_client()
client

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.5eccf95362ee408bbaae0e717289b506 Dashboard: /services/dask-gateway/clusters/prod.5eccf95362ee408bbaae0e717289b506/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
from xhistogram.xarray import histogram
import xarray as xr
import numpy as np
import dask.array as dsa

def pdf_wrapper(ds, bins_dict={'thetao':np.linspace(-4, 46, 2000), 'so':np.linspace(0, 50, 2000)}, dim=['x', 'y', 'lev']):
    
    variables = list(bins_dict.keys())
    
    missing_vars = [var for var in variables if var not in ds.data_vars]
    if len(missing_vars) > 0:
        raise ValueError(f'Bins defined for {missing_vars} in `bins_dict` could not be found in dataset variables {list(ds.data_vars)}')
    
        
    ds = ds.squeeze()
    args = tuple([ds[v] for v in variables])
    bins = [bins_dict[v] for v in variables]
    bin_names = [v+'_bin' for v in variables]
    
    vol = ds.areacello * ds.thkcello

    hist = histogram(*args, bins=bins, weights=vol, dim=dim)
    pdf = hist#/hist.sum(bin_names) # normalize by the full volume
    return pdf

In [4]:
# lets create an artificial dataset for testing (this is on the lower end of resolutions for the CMIP6 data)

nx,ny,nz,nt = 360, 291, 45, 1980
# first try it with consistent chunk sizes (the actual datases can have slightly different time chunks)
cx,cy,cz,ct = nx, ny, nz, 1

so = xr.DataArray(dsa.random.random([nx,ny,nz,nt], chunks=[cx,cy,cz,ct]), dims=['x', 'y', 'lev', 'time'])
thetao = xr.DataArray(dsa.random.random([nx,ny,nz,nt], chunks=[cx,cy,cz,ct]), dims=['x', 'y', 'lev', 'time'])
areacello = xr.DataArray(dsa.random.random([nx,ny], chunks=[cx,cy]), dims=['x', 'y'])
thkcello = xr.DataArray(dsa.random.random([nx,ny,nz,nt], chunks=[cx,cy,cz,ct]), dims=['x', 'y', 'lev', 'time'])
llon, llat = np.meshgrid(np.linspace(0,360, nx), np.linspace(-90,90, ny), indexing='ij')
lon = xr.DataArray(llon, dims=['x','y'])
lat = xr.DataArray(llat, dims=['x','y'])


ds_synth = xr.Dataset({'thetao':thetao, 'so':so})
ds_synth = ds_synth.assign_coords(areacello=areacello, thkcello=thkcello, lon=lon, lat=lat)

# ds_synth = 
ds_synth

<xarray.Dataset>
Dimensions:    (lev: 45, time: 1980, x: 360, y: 291)
Coordinates:
    areacello  (x, y) float64 dask.array<chunksize=(360, 291), meta=np.ndarray>
    thkcello   (x, y, lev, time) float64 dask.array<chunksize=(360, 291, 45, 1), meta=np.ndarray>
    lon        (x, y) float64 0.0 0.0 0.0 0.0 0.0 ... 360.0 360.0 360.0 360.0
    lat        (x, y) float64 -90.0 -89.38 -88.76 -88.14 ... 88.76 89.38 90.0
Dimensions without coordinates: lev, time, x, y
Data variables:
    thetao     (x, y, lev, time) float64 dask.array<chunksize=(360, 291, 45, 1), meta=np.ndarray>
    so         (x, y, lev, time) float64 dask.array<chunksize=(360, 291, 45, 1), meta=np.ndarray>

In [5]:
from dask.distributed import performance_report

with performance_report(filename="2d-histogram-dask-report.html"):
    out = pdf_wrapper(ds_synth).load()

distributed.client - WARNING - Couldn't gather 1980 keys, rescheduling {"('getitem-6780abf48d484c0609f0da59fde5480a', 4, 0, 0)": ('tls://10.37.44.14:45603',), "('getitem-6780abf48d484c0609f0da59fde5480a', 755, 0, 0)": ('tls://10.37.45.14:38401',), "('getitem-6780abf48d484c0609f0da59fde5480a', 240, 0, 0)": ('tls://10.37.42.12:40273',), "('getitem-6780abf48d484c0609f0da59fde5480a', 274, 0, 0)": ('tls://10.37.2.32:42015',), "('getitem-6780abf48d484c0609f0da59fde5480a', 1230, 0, 0)": ('tls://10.37.42.12:40273',), "('getitem-6780abf48d484c0609f0da59fde5480a', 320, 0, 0)": ('tls://10.37.42.12:40273',), "('getitem-6780abf48d484c0609f0da59fde5480a', 837, 0, 0)": ('tls://10.37.21.15:40427',), "('getitem-6780abf48d484c0609f0da59fde5480a', 585, 0, 0)": ('tls://10.37.47.5:43751',), "('getitem-6780abf48d484c0609f0da59fde5480a', 352, 0, 0)": ('tls://10.37.42.12:40273',), "('getitem-6780abf48d484c0609f0da59fde5480a', 1905, 0, 0)": ('tls://10.37.47.5:43751',), "('getitem-6780abf48d484c0609f0da59fde548

KilledWorker: ("('random_sample-transpose-reshape-551fcfb7674f252ae0a1da5e40d61bd8', 1526, 0)", <Worker 'tls://10.37.2.32:42015', name: dask-worker-5eccf95362ee408bbaae0e717289b506-4jdxc, memory: 0, processing: 4700>)